In [ ]:
pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 87.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 90.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.9 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=a1cc7288ca7e0a29fd6f7e139c3c4d2ca152c6c3bb061c2bfbfafef38ffd86a4
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.question_answering import QuestionAnsweringModel

In [ ]:
# Load dataset
df = pd.read_csv("/kaggle/input/comprehensive-medical-q-a-dataset/train.csv")

# Split dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

# Convert data to required format for Simple Transformers
train_data = []
for index, row in train_df.iterrows():
    train_data.append({"qas_id": str(index), "question": row["Question"], "context": row["Answer"], "answers": {"text": [row["Answer"]], "answer_start": [0]}})

test_data = []
for index, row in test_df.iterrows():
    test_data.append({"qas_id": str(index), "question": row["Question"], "context": row["Answer"], "answers": {"text": [row["Answer"]], "answer_start": [0]}})

df.head()

,qtype,Question,Answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."


In [ ]:
train_data = []
for index, row in train_df.iterrows():
    train_data.append({"context": row["Answer"], "qas": [{"id": str(index), "question": row["Question"], "answers": [{"text": row["Answer"], "answer_start": 0}]}]})

test_data = []
for index, row in test_df.iterrows():
    test_data.append({"context": row["Answer"], "qas": [{"id": str(index), "question": row["Question"], "answers": [{"text": row["Answer"], "answer_start": 0}]}]})


train_data = train_data[0:5000]
print(len(train_data))
print(len(test_data))

5000
4102


In [ ]:
bert_args = {
    "overwrite_output_dir": True,
    "max_seq_length": 512,
    "num_train_epochs": 3,
    "train_batch_size": 8,
    "learning_rate": 3e-5,
    "doc_stride": 128,
    "reprocess_input_data": True,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 1000,
    "evaluate_during_training_verbose": True,
    "use_cached_eval_features": False,
    "save_eval_checkpoints": False,
    "wandb_project": "medquad-bert",
    "use_cuda": True  # Set use_cuda to False
}

bert_model = QuestionAnsweringModel("bert", "bert-base-uncased", args=bert_args)
bert_model.train_model(train_data, eval_data=test_data)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
add example index and unique id: 100%|██████████| 5000/5000 [00:00<00:00, 496273.37it/s]


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Training loss,█▃▂▁▁
global_step,▁▃▅▆█
lr,▁▃▅▆█
Training loss,0.01173
global_step,250
lr,2e-05


Running Epoch 1 of 3:   0%|          | 0/870 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 4102/4102 [00:37<00:00, 110.21it/s]

add example index and unique id: 100%|██████████| 4102/4102 [00:00<00:00, 518974.27it/s]


...



convert squad examples to features:   0%|          | 0/4102 [00:00<?, ?it/s]

convert squad examples to features:   0%|          | 1/4102 [00:18<20:58:18, 18.41s/it]

convert squad examples to features:  25%|██▌       | 1026/4102 [00:19<00:41, 74.90it/s]

convert squad examples to features:  25%|██▌       | 1026/4102 [00:31<00:41, 74.90it/s]

convert squad examples to features:  50%|█████     | 2051/4102 [00:35<00:30, 67.09it/s]

convert squad examples to features: 100%|██████████| 4102/4102 [00:37<00:00, 109.62it/s]


add example index and unique id: 100%|██████████| 4102/4102 [00:00<00:00, 518317.62it/s]


Running Evaluation:   0%|          | 0/56 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 4102/4102 [00:37<00:00, 108.33it/s]

add example index and unique id: 100%|██████████| 4102/4102 [00:00<00:00, 511065.94it/s]


Running Evaluation:   0%|          | 0/56 [00:00<?, ?it/s]

(2610,
 {'global_step': [870, 1000, 1740, 2000, 2610],
  'correct': [818, 818, 818, 818, 818],
  'similar': [2945, 2946, 2947, 2947, 2947],
  'incorrect': [339, 338, 337, 337, 337],
  'train_loss': [0.004629765171557665,
   0.00105392187833786,
   0.0002079265541397035,
   0.0006839260458946228,
   0.00012378182145766914],
  'eval_loss': [-7.654994419642857,
   -7.425013950892857,
   -7.918038504464286,
   -7.787388392857143,
   -7.708426339285714]})

In [ ]:
# Fine-tune MobileBERT
mobilebert_args = {
    "overwrite_output_dir": True,
    "max_seq_length": 512,
    "num_train_epochs": 3,
    "train_batch_size": 8,
    "learning_rate": 3e-5,
    "doc_stride": 128,
    "reprocess_input_data": True,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 1000,
    "evaluate_during_training_verbose": True,
    "use_cached_eval_features": False,
    "save_eval_checkpoints": False,
    "wandb_project": "medquad-mobilebert",
    "use_cuda": True
}

mobilebert_model = QuestionAnsweringModel("mobilebert", "google/mobilebert-uncased", args=mobilebert_args)
mobilebert_model.train_model(train_data, eval_data=test_data)


config.json:   0%|          | 0.00/847 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/147M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of MobileBertForQuestionAnswering were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...



convert squad examples to features:   0%|          | 0/4102 [00:00<?, ?it/s]

convert squad examples to features:   0%|          | 1/4102 [00:18<21:04:55, 18.51s/it]

convert squad examples to features:  25%|██▌       | 1026/4102 [00:19<00:42, 73.22it/s]

convert squad examples to features:  50%|█████     | 2051/4102 [00:34<00:29, 69.40it/s]

convert squad examples to features: 100%|██████████| 4102/4102 [00:36<00:00, 111.08it/s]


add example index and unique id: 100%|██████████| 4102/4102 [00:00<00:00, 516869.50it/s]


Running Evaluation:   0%|          | 0/56 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 4102/4102 [00:36<00:00, 111.07it/s]

add example index and unique id: 100%|██████████| 4102/4102 [00:00<00:00, 514289.32it/s]


Running Evaluation:   0%|          | 0/56 [00:00<?, ?it/s]

(2610,
 {'global_step': [870, 1000, 1740, 2000, 2610],
  'correct': [0, 0, 0, 0, 0],
  'similar': [4102, 4102, 4102, 4102, 4102],
  'incorrect': [0, 0, 0, 0, 0],
  'train_loss': [nan, nan, nan, nan, nan],
  'eval_loss': [nan, nan, nan, nan, nan]})

In [ ]:
# Fine-tune RoBERTa
roberta_args = {
    "overwrite_output_dir": True,
    "max_seq_length": 512,
    "num_train_epochs": 3,
    "train_batch_size": 8,
    "learning_rate": 3e-5,
    "doc_stride": 128,
    "reprocess_input_data": True,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 1000,
    "evaluate_during_training_verbose": True,
    "use_cached_eval_features": False,
    "save_eval_checkpoints": False,
    "wandb_project": "medquad-roberta",
    "use_cuda": True
}

roberta_model = QuestionAnsweringModel("roberta", "roberta-base", args=roberta_args)
roberta_model.train_model(train_data, eval_data=test_data)

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
convert squad examples to features:   0%|          | 0/5000 [01:44<?, ?it/s]Process ForkPoolWorker-30:
Process ForkPoolWorker-29:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", 

KeyboardInterrupt: 

In [ ]:
result, texts = bert_model.eval_model(test_data)

add example index and unique id: 100%|██████████| 4102/4102 [00:00<00:00, 493023.33it/s]


Running Evaluation:   0%|          | 0/56 [00:00<?, ?it/s]

In [ ]:
#Hyper Tunning
# Define hyperparameters to tune
hyperparameters = {
    "dropout_rate": [0.3, 0.5, 0.7],
    "num_hidden_layers": [6, 8, 12],
    "learning_rate": [1e-5, 2e-5, 3e-5]
}

best_f1_score = 0
best_hyperparameters = None

# Loop through all combinations of hyperparameters
for dropout_rate in hyperparameters["dropout_rate"]:
    for num_hidden_layers in hyperparameters["num_hidden_layers"]:
        for learning_rate in hyperparameters["learning_rate"]:
            # Define model with current hyperparameters
            model_args = {
                "dropout_rate": dropout_rate,
                "num_hidden_layers": num_hidden_layers,
                "learning_rate": learning_rate,
            }

            # Fine-tune model with current hyperparameters
            model = QuestionAnsweringModel("bert", "bert-base-uncased", args=model_args)
            model.train_model(train_data, eval_data=test_data)

            result, _ = model.eval_model(test_data)
            f1_score = result["f1"]

            if f1_score > best_f1_score:
                best_f1_score = f1_score
                best_hyperparameters = model_args

print("Best Hyperparameters:", best_hyperparameters)
print("Best F1 Score:", best_f1_score)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Output directory (outputs/) already exists and is not empty.Use --overwrite_output_dir to overcome.